In [85]:
import requests, json
import pandas as pd
import numpy as np
from pprint import pprint
import requests

pd.options.mode.chained_assignment = None  # default='warn'

In [12]:
print(os.getcwd())

/Users/trajche/Desktop/python/FPL_Data_Pull


In [23]:
#Logging in and getting my team
#Do NOT run with VPN on :) 

s = requests.Session()

headers = {
   'authority': 'users.premierleague.com' ,
   'cache-control': 'max-age=0' ,
   'upgrade-insecure-requests': '1' ,
   'origin': 'https://fantasy.premierleague.com' ,
   'content-type': 'application/x-www-form-urlencoded' ,
   'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36' ,
   'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' ,
   'sec-fetch-site': 'same-site' ,
   'sec-fetch-mode': 'navigate' ,
   'sec-fetch-user': '?1' ,
   'sec-fetch-dest': 'document' ,
   'referer': 'https://fantasy.premierleague.com/my-team' ,
   'accept-language': 'en-US,en;q=0.9,he;q=0.8' ,
}
with open("/Users/trajche/Desktop/python/FPL_Data_Pull/credentials.json", 'r') as f:
  data = json.load(f)


url = "https://users.premierleague.com/accounts/login/"

res = s.post(url, data = data,  headers = headers)

team_url = "https://fantasy.premierleague.com/api/my-team/203128/"
res = s.get(team_url)
json.loads(res.content)


{'picks': [{'element': 113,
   'position': 1,
   'selling_price': 45,
   'multiplier': 1,
   'purchase_price': 45,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 285,
   'position': 2,
   'selling_price': 75,
   'multiplier': 1,
   'purchase_price': 75,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 306,
   'position': 3,
   'selling_price': 70,
   'multiplier': 1,
   'purchase_price': 70,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 16,
   'position': 4,
   'selling_price': 50,
   'multiplier': 1,
   'purchase_price': 50,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 146,
   'position': 5,
   'selling_price': 60,
   'multiplier': 1,
   'purchase_price': 60,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 312,
   'position': 6,
   'selling_price': 60,
   'multiplier': 1,
   'purchase_price': 60,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 283,
   'position'

In [24]:
s = json.loads(res.content)

In [66]:
#Getting my current team
myteam = pd.json_normalize(s['picks'])
print(myteam.head(5))

   element  position  selling_price  multiplier  purchase_price  is_captain  \
0      113         1             45           1              45       False   
1      285         2             75           1              75       False   
2      306         3             70           1              70       False   
3       16         4             50           1              50       False   
4      146         5             60           1              60       False   

   is_vice_captain  
0            False  
1            False  
2            False  
3            False  
4            False  


In [78]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 8058892}


In [91]:
players = pd.DataFrame(r['elements'])
print(players.columns)

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [92]:
#Getting the required columns
players_condensed = players[['id', 'web_name', 'team', 'element_type', 'now_cost', 'selected_by_percent', 'event_points', 'total_points']]
players_condensed.head(5)

,id,web_name,team,element_type,now_cost,selected_by_percent,event_points,total_points
0,1,Cédric,1,2,45,0.3,0,0
1,3,Xhaka,1,3,50,0.7,2,2
2,4,Elneny,1,3,45,1.1,0,0
3,5,Holding,1,2,45,0.2,0,0
4,6,Partey,1,3,50,0.6,3,3


In [93]:
elements_keys = pd.DataFrame(r['element_types'])
teams_keys = pd.DataFrame(r['teams'])

Map the team, element_type, position and join with myteam

In [94]:
players_condensed['position'] = players_condensed.element_type.map(elements_keys.set_index('id').singular_name)
players_condensed.head(5)

,id,web_name,team,element_type,now_cost,selected_by_percent,event_points,total_points,position
0,1,Cédric,1,2,45,0.3,0,0,Defender
1,3,Xhaka,1,3,50,0.7,2,2,Midfielder
2,4,Elneny,1,3,45,1.1,0,0,Midfielder
3,5,Holding,1,2,45,0.2,0,0,Defender
4,6,Partey,1,3,50,0.6,3,3,Midfielder


In [95]:
players_condensed['team_name'] = players_condensed.team.map(teams_keys.set_index('id').name)
players_condensed

,id,web_name,team,element_type,now_cost,selected_by_percent,event_points,total_points,position,team_name
0,1,Cédric,1,2,45,0.3,0,0,Defender,Arsenal
1,3,Xhaka,1,3,50,0.7,2,2,Midfielder,Arsenal
2,4,Elneny,1,3,45,1.1,0,0,Midfielder,Arsenal
3,5,Holding,1,2,45,0.2,0,0,Defender,Arsenal
4,6,Partey,1,3,50,0.6,3,3,Midfielder,Arsenal
...,...,...,...,...,...,...,...,...,...,...
554,494,Mosquera,20,2,40,1.0,0,0,Defender,Wolves
555,503,Moutinho,20,3,50,2.1,0,0,Midfielder,Wolves
556,516,Collins,20,2,45,0.3,0,0,Defender,Wolves
557,557,Smith,20,1,40,0.5,0,0,Goalkeeper,Wolves


Joining myteam with players_condensed to map out my team with the keys

In [101]:
myteam_df = pd.merge(myteam, players_condensed, how="left", left_on= "element", right_on= 'id')

In [102]:
myteam_df

,element,position_x,selling_price,multiplier,purchase_price,is_captain,is_vice_captain,id,web_name,team,element_type,now_cost,selected_by_percent,event_points,total_points,position_y,team_name
0,113,1,45,1,45,False,False,113,Sánchez,5,1,45,11.2,0,0,Goalkeeper,Brighton
1,285,2,75,1,75,False,False,285,Alexander-Arnold,12,2,75,59.7,0,0,Defender,Liverpool
2,306,3,70,1,70,False,False,306,Cancelo,13,2,70,45.3,0,0,Defender,Man City
3,16,4,50,1,50,False,False,16,Gabriel,1,2,50,8.7,7,7,Defender,Arsenal
4,146,5,60,1,60,False,False,146,James,6,2,60,40.7,0,0,Defender,Chelsea
5,312,6,60,1,60,False,False,312,Dias,13,2,60,11.9,0,0,Defender,Man City
6,283,7,130,1,130,False,True,283,Salah,12,3,130,61.7,0,0,Midfielder,Liverpool
7,293,8,80,1,80,False,False,293,Luis Díaz,12,3,80,21.5,0,0,Midfielder,Liverpool
8,19,9,60,1,60,False,False,19,Martinelli,1,3,60,22.3,8,8,Midfielder,Arsenal
9,28,10,80,1,80,False,False,28,Jesus,1,4,80,76.4,2,2,Forward,Arsenal


In [104]:
myteam_final = myteam_df[["id", "web_name", "position_y", "team_name", "is_captain", "now_cost", "selling_price", "selected_by_percent", "event_points", "total_points"]]
myteam_final.head()

,id,web_name,position_y,team_name,is_captain,now_cost,selling_price,selected_by_percent,event_points,total_points
0,113,Sánchez,Goalkeeper,Brighton,False,45,45,11.2,0,0
1,285,Alexander-Arnold,Defender,Liverpool,False,75,75,59.7,0,0
2,306,Cancelo,Defender,Man City,False,70,70,45.3,0,0
3,16,Gabriel,Defender,Arsenal,False,50,50,8.7,7,7
4,146,James,Defender,Chelsea,False,60,60,40.7,0,0


Getting the top level team information

In [109]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'entry/203128/history/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{'chips': [], 'current': [...], 'past': [...]}


In [111]:
team_info = pd.DataFrame(r['current'])
team_info

,event,points,total_points,rank,rank_sort,overall_rank,bank,value,event_transfers,event_transfers_cost,points_on_bench
0,1,40,40,488858,490960,488859,0,1000,0,0,2


In [114]:
team_history = pd.DataFrame(r['past'])
team_history

,season_name,total_points,rank
0,2013/14,1409,2784018
1,2014/15,1581,2194409
2,2015/16,1701,2213869
3,2016/17,1646,2819685
4,2017/18,2021,1013063
5,2018/19,2222,202002
6,2019/20,2375,5952
7,2020/21,2319,291454
8,2021/22,2601,18278
